In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

Using TensorFlow backend.


In [2]:
department = pd.read_csv("/home/tinku/Downloads/sptm/data/document_departments.csv")

In [3]:
# this finds our json files
path_to_json = '/home/tinku/Downloads/sptm/data/docs/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['_id', 'jd_information'])

# we need both the json and an index number so use enumerate()
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = pd.read_json(json_file)
        #print(json_text.head())
        
        
        temp = json_text[['_id', 'jd_information']].loc[["description"]]
        jsons_data = jsons_data.append(temp)
        if (( index % 200 ) == 0):
            print(index)
    

0
200
400
600
800
1000


In [4]:
department.head()

,Document ID,Department
0,8196431,Customer service
1,8278123,Digital Marketing
2,8331625,Sales
3,8331003,Back office ticketing
4,8277939,Digital Marketing


In [5]:
jsons_data.head()

,_id,jd_information
description,6975593,We have an urgent opening for&nbsp;Customer ca...
description,8044547,We have urgent opening for profile of&nbsp;Tou...
description,8136539,The primary role of the TSM/TSO is to prospect...
description,8131576,
description,8343612,


In [6]:
department.Department.nunique()

30

In [7]:
department.columns

Index(['Document ID', 'Department'], dtype='object')

In [8]:
department.columns = ['_id', 'Department']

In [9]:
department.head()

,_id,Department
0,8196431,Customer service
1,8278123,Digital Marketing
2,8331625,Sales
3,8331003,Back office ticketing
4,8277939,Digital Marketing


In [10]:
jsons_data.head()

,_id,jd_information
description,6975593,We have an urgent opening for&nbsp;Customer ca...
description,8044547,We have urgent opening for profile of&nbsp;Tou...
description,8136539,The primary role of the TSM/TSO is to prospect...
description,8131576,
description,8343612,


In [11]:
jsons_data.reset_index(drop=True, inplace=True)

In [12]:
jsons_data.head()

,_id,jd_information
0,6975593,We have an urgent opening for&nbsp;Customer ca...
1,8044547,We have urgent opening for profile of&nbsp;Tou...
2,8136539,The primary role of the TSM/TSO is to prospect...
3,8131576,
4,8343612,


In [13]:
jsons_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 2 columns):
_id               1162 non-null object
jd_information    1162 non-null object
dtypes: object(2)
memory usage: 18.2+ KB


In [14]:
department.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 2 columns):
_id           1162 non-null int64
Department    1162 non-null object
dtypes: int64(1), object(1)
memory usage: 18.2+ KB


In [15]:
jsons_data['_id'] = jsons_data['_id'].astype(str).astype(int)

In [16]:
jsons_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 2 columns):
_id               1162 non-null int64
jd_information    1162 non-null object
dtypes: int64(1), object(1)
memory usage: 18.2+ KB


In [17]:
data = pd.merge(left = jsons_data, right=department, on = '_id', how = 'left')

In [18]:
data.head()

,_id,jd_information,Department
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales
3,8131576,,Finance
4,8343612,,Sales


In [19]:
tt = data.copy()

In [20]:
tt.shape

(1162, 3)

In [21]:
tt.head()

,_id,jd_information,Department
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales
3,8131576,,Finance
4,8343612,,Sales


In [22]:
tt.replace(r'^\s*$', np.nan, regex=True, inplace = True)

In [23]:
tt.head()

,_id,jd_information,Department
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales
3,8131576,NaN,Finance
4,8343612,NaN,Sales


In [24]:
tt.isnull().sum()

_id                 0
jd_information    417
Department          0
dtype: int64

In [25]:
tt.dropna(inplace=True)

In [26]:
tt.shape

(745, 3)

In [27]:
tt.head()

,_id,jd_information,Department
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales
5,8297065,No Consultancy Charges &ndash;Call Gunjan -958...,Ticketing
7,8207767,We need committed &amp; result-oriented people...,Digital Marketing


In [28]:
tt.reset_index(drop=True, inplace=True)

In [29]:
tt.head()

,_id,jd_information,Department
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales
3,8297065,No Consultancy Charges &ndash;Call Gunjan -958...,Ticketing
4,8207767,We need committed &amp; result-oriented people...,Digital Marketing


In [30]:
tt.Department.value_counts()

Ticketing                          230
Sales                              148
Digital Marketing                  132
Customer service                    49
Marketing                           30
Finance                             26
Operations                          25
Back office ticketing               16
Administration                      15
IT                                  13
Presales                            10
Technology                          10
Maintenance                          8
Recruitment                          5
Public Relations                     5
Engineering Design Construction      4
Logistics                            3
Marine Engineering                   3
Marine Deck                          2
Content                              2
Data entry                           2
Analytics                            2
Procurement                          1
Marine Service Steward               1
Airline Ground Operations            1
Flight Operations        

In [31]:
tt['ncount'] = tt.groupby(['Department'])['Department'].transform('count')

In [32]:
tt.head()

,_id,jd_information,Department,ncount
0,6975593,We have an urgent opening for&nbsp;Customer ca...,Ticketing,230
1,8044547,We have urgent opening for profile of&nbsp;Tou...,Ticketing,230
2,8136539,The primary role of the TSM/TSO is to prospect...,Sales,148
3,8297065,No Consultancy Charges &ndash;Call Gunjan -958...,Ticketing,230
4,8207767,We need committed &amp; result-oriented people...,Digital Marketing,132


In [33]:
tt = tt[tt.ncount > 1]

In [34]:
tt.drop(['ncount'], inplace=True, axis = 1)

In [35]:
train = tt.copy()

In [36]:
from sklearn.preprocessing import LabelEncoder
lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(train.Department.values)

In [37]:
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(train.jd_information.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [38]:
print (xtrain.shape)
print (xvalid.shape)

(592,)
(148,)


In [39]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [40]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [41]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.521 


In [42]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)


In [43]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 1.182 


In [44]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.512 


In [45]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 6.807 


In [46]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [47]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))



logloss: 1.323 


In [48]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.219 


In [49]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 1.171 


In [50]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 1.150 
